### Camada Gold Delta
#### Diferenças entre a camada Gold e Gold Delta

##### Camada Gold:

- Representa a camada final de um Data Lake ou Data Warehouse.
- Aqui os dados são refinados e estruturados em tabelas de fatos e dimensões para suportar análises e relatórios.
- As tabelas são otimizadas para desempenho e organizadas em modelos como Star Schema ou Snowflake Schema.


##### Gold Delta:

- Utiliza o formato Delta Lake, que combina as vantagens de sistemas transacionais (ACID) e data lakes escaláveis.
- Oferece versionamento, controle de concorrência e suporte eficiente para consultas de larga escala.

##### Star Schema (Esquema Estrela):

- Modelo simples de banco de dados usado em data warehouses.
- Estrutura: Uma tabela fato central conectada diretamente a tabelas dimensão (ex.: produtos, clientes).
- Vantagens: Consultas rápidas e fácil entendimento.
- Desvantagem: Dados redundantes nas dimensões.


##### Snowflake Schema (Esquema Floco de Neve):

- Versão mais normalizada do Star Schema.
- Dimensões divididas em subdimensões (ex.: informações de região separadas da dimensão cliente).
- Vantagens: Menor redundância e uso otimizado de armazenamento.
- Desvantagem: Estrutura mais complexa e consultas mais lentas devido ao aumento de joins.

##### Diferença Principal entre Star Schema e Snowflake Schema:

- Star Schema é mais rápido para consultas e análises simples.
- Snowflake Schema é mais eficiente em armazenamento e mantém maior consistência nos dados.

##### Tabelas Fato:

- Registram eventos ou transações, como vendas ou cliques.
- Contêm métricas numéricas (ex.: quantidade, valor total) e chaves que se conectam às dimensões.
- Foco em análise, cálculos e tendências.

##### Tabelas Dimensão:

- Guardam dados descritivos, como informações sobre clientes, produtos ou regiões.
- Complementam a tabela fato, permitindo análises por diferentes perspectivas.
- Exemplos: detalhes de produtos, dados demográficos de clientes.

##### Relação entre Tabelas Fato e Dimensão: 

As tabelas fato utilizam chaves para se conectar às dimensões, formando um modelo que facilita análises multidimensionais, como vendas por região ou por categoria de produto.


### Configuração Inicial para Delta Lake
O código abaixo configura o ambiente para trabalhar com o Delta Lake em um pipeline de dados. O Delta Lake é uma camada de armazenamento que oferece transações ACID, versionamento e otimização para análises em grande escala. Aqui, os dados são lidos da Camada Silver, que contém informações já refinadas, e estarão prontos para serem processados na Camada Gold.

In [0]:
# 1. Importar o SparkSession para inicializar e configurar o ambiente de trabalho.
from pyspark.sql import SparkSession

# 2. Configurar a SparkSession para habilitar o suporte ao Delta Lake.
spark = SparkSession.builder \
    .appName("Carga Delta") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Explicação do código acima:
# appName: Nome da aplicação Spark para identificação nos logs.
# spark.sql.extensions: Extensão necessária para uso do Delta Lake.
# spark.sql.catalog.spark_catalog: Configura o catálogo do Delta.
# getOrCreate: Cria a sessão Spark.

# 3. Definir os caminhos de entrada e saída no Data Lake:
# - `silver_path`: Localização dos dados refinados, oriundos da camada Silver.
# - `gold_path`: Destino para armazenar os dados processados na camada Gold, no formato Delta.
silver_path = "/mnt/lhdw/silver/vendas"  # Caminho para os dados refinados da Silver.
gold_path = "/mnt/lhdw/gold/vendas_delta"  # Caminho para armazenamento na Gold.

# 4. Ler os dados da Camada Silver:
# Aqui, os dados são carregados no formato Parquet para preparação de transformações ou agregações.
df_silver = spark.read.format("parquet").load(silver_path)  # Lê os dados da camada Silver em um DataFrame.




#### Criação da Dimensão Produto

Nesta etapa, criamos a dimensão Produto, um componente essencial para um modelo dimensional no formato estrela (star schema). A dimensão contém atributos únicos relacionados aos produtos, como ID, nome e categoria. Além disso, adicionamos uma chave substituta (surrogate key), que é uma prática comum em data warehouses para garantir unicidade e eficiência.

In [0]:
# 1. Importar a função para gerar chaves substitutas (monotonicamente crescentes).
from pyspark.sql.functions import monotonically_increasing_id

# 2. Definir o nome da tabela de destino para melhor organização no Data Lake.
tb_destino = "dim_produto"  # Nome da tabela de dimensão para identificar produtos.

# 3. Extrair informações únicas de produtos:
# Selecionamos as colunas relevantes para a dimensão Produto e removemos duplicidades.
dim_produto_df = df_silver.select(
    "IDProduto", "Produto", "Categoria"  # Atributos necessários para a dimensão.
).dropDuplicates()  # Garante que cada combinação de Produto seja única.

# 4. Adicionar uma chave substituta (surrogate key):
# A chave substituta é gerada para cada linha, começando a partir de 1.
dim_produto_df = dim_produto_df.withColumn("sk_produto", monotonically_increasing_id() + 1)

# 5. Escrever a tabela de dimensão no formato Delta:
# O formato Delta é usado para suporte a transações ACID e otimização de consultas.
dim_produto_df.write.format("delta") \
               .mode("overwrite") \
               .save(f"{gold_path}/{tb_destino}")

# Explicação do código acima:
# format("delta"): Define o formato de saída como Delta.
# Substitui os dados existentes, se houver.
# Salva no caminho da camada Gold.

# Código adicional para verificação:
display(dim_produto_df)  # Exibe os dados processados para validação.
dim_produto_df.count()  # Conta o número de registros na dimensão.



IDProduto,Produto,Categoria,sk_produto
585,Maximus UC-50,Urban,1
555,Maximus UC-20,Mix,2
423,Maximus UM-28,Urban,3
681,Maximus UC-46,Urban,4
628,Maximus UC-93,Urban,5
415,Maximus UM-20,Urban,6
547,Maximus UC-12,Mix,7
653,Maximus UC-18,Urban,8
512,Maximus UR-01,Urban,9
520,Maximus UE-08,Urban,10


Out[3]: 118

#### Criação da Dimensão Categoria

Nesta etapa, criamos a dimensão Categoria, que serve como uma tabela auxiliar para organizar as categorias únicas presentes nos dados. Cada categoria é identificada por uma chave substituta (surrogate key), adicionada para facilitar a integração e otimização em modelos dimensionais.

In [0]:
# 1. Importar a função necessária para gerar chaves substitutas.
from pyspark.sql.functions import monotonically_increasing_id

# 2. Definir o nome da tabela de destino para organização no Data Lake.
tb_destino = "dim_categoria"  # Nome da tabela de dimensão para categorias.

# 3. Extrair categorias únicas:
# Selecionamos a coluna "Categoria" do conjunto de dados e removemos duplicatas para criar uma lista única.
dim_categoria_df = df_silver.select(
    "Categoria"  # Coluna necessária para a dimensão.
).dropDuplicates()  # Garante que cada categoria apareça apenas uma vez.

# 4. Adicionar uma chave substituta (surrogate key):
# A chave substituta é criada para cada linha, começando a partir de 1.
dim_categoria_df = dim_categoria_df.withColumn("sk_categoria", monotonically_increasing_id() + 1)

# 5. Escrever a tabela de dimensão no formato Delta:
# Usamos o formato Delta para maior robustez e otimização, substituindo dados existentes.
dim_categoria_df.write.format("delta") \
                 .mode("overwrite") \
                 .save(f"{gold_path}/{tb_destino}")

# Explicação do código acima:
# format("delta"): Define o formato de saída como Delta.
# Substitui os dados existentes, se houver.
# Salva no caminho da camada Gold.

# 6. Contar os registros da dimensão:
# Realiza uma contagem de registros para validação.
dim_categoria_df.count()  # Conta o número de categorias únicas processadas.


Out[4]: 5

#### Criação da Dimensão Segmento

Nesta etapa, criamos a dimensão Segmento, que contém os segmentos únicos dos dados. A principal função dessa tabela é organizar os dados por segmentos específicos, facilitando consultas e análises relacionadas a esses grupos. Para garantir que cada segmento tenha uma identificação única, adicionamos uma chave substituta (surrogate key). Essa chave facilita a integridade dos dados e a integração com outras tabelas no modelo dimensional.

In [0]:
# 1. Definir o nome da tabela de destino para a dimensão Segmento.
tb_destino = "dim_segmento"  # Nome da tabela de dimensão para os segmentos.

# 2. Extrair segmentos únicos:
# Selecionamos a coluna "Segmento" do DataFrame df_silver e removemos as duplicatas para garantir que cada segmento apareça apenas uma vez.
dim_segmento_df = df_silver.select(
   "Segmento"  # Coluna necessária para a dimensão.
).dropDuplicates()  # Remove duplicatas, garantindo que cada segmento seja único.

# 3. Adicionar chave substituta (surrogate key):
# A chave substituta é gerada para garantir que cada linha tenha uma identidade única, começando de 1.
dim_segmento_df = dim_segmento_df.withColumn("sk_segmento", monotonically_increasing_id() + 1)

# 4. Escrever a tabela de dimensão no formato Delta:
# Usamos o formato Delta para garantir consistência e otimização na leitura/escrita de dados.
dim_segmento_df.write.format("delta") \
                 .mode("overwrite") \
                 .save(f"{gold_path}/{tb_destino}")

# Explicação do código acima:
# format("delta"): Define o formato de saída como Delta.
# Substitui os dados existentes, se houver.
# Salva no caminho da camada Gold.

# 5. Contar o número de registros na dimensão:
# Realiza uma contagem para verificar o número de segmentos únicos processados.
dim_segmento_df.count()  # Conta o número de segmentos únicos na dimensão.


Out[5]: 9

#### Criação da Dimensão Fabricante

Nesta etapa, criamos a dimensão Fabricante, que armazena informações sobre os fabricantes dos produtos. Essa tabela é essencial para a análise detalhada de vendas e a compreensão das contribuições de diferentes fabricantes para o desempenho das vendas. Assim como nas outras dimensões, aplicamos uma chave substituta (surrogate key) para garantir uma identidade única para cada fabricante, facilitando a integração com as outras tabelas do modelo dimensional.

In [0]:
# 1. Definir o nome da tabela de destino para a dimensão Fabricante.
tb_destino = "dim_fabricante"  # Nome da tabela de dimensão para os fabricantes.

# 2. Extrair fabricantes únicos:
# Selecionamos as colunas "IDFabricante" e "Fabricante" do DataFrame df_silver,
# garantindo que apenas os fabricantes únicos sejam extraídos.
dim_fabricante_df = df_silver.select(
    "IDFabricante", "Fabricante"  # Colunas necessárias para a dimensão de fabricantes.
).dropDuplicates()  # Remove duplicatas, garantindo que cada fabricante seja único.

# 3. Adicionar chave substituta (surrogate key):
# A chave substituta é gerada para garantir que cada fabricante tenha uma identidade única.
dim_fabricante_df = dim_fabricante_df.withColumn("sk_fabricante", monotonically_increasing_id() + 1)

# 4. Escrever a tabela de dimensão no formato Delta:
# Utilizamos o formato Delta para garantir integridade transacional e otimização no desempenho de leitura/escrita.
dim_fabricante_df.write.format("delta") \
                 .mode("overwrite") \
                 .save(f"{gold_path}/{tb_destino}")

# Explicação do código acima:
# format("delta"): Define o formato de saída como Delta.
# Substitui os dados existentes, se houver.
# Salva no caminho da camada Gold.

# 5. Contar o número de registros na dimensão:
# Realiza uma contagem para verificar quantos fabricantes foram processados.
dim_fabricante_df.count()  # Conta o número de fabricantes únicos na dimensão.


Out[6]: 1

#### Criação da Dimensão Geografia

Nesta etapa, estamos criando a dimensão Geografia, que armazena informações relacionadas ao local geográfico dos dados, como cidade, estado, região, distrito, país e código postal. Essa tabela é fundamental para análises que envolvem a localização geográfica, como estudos regionais de vendas ou distribuição. Assim como nas outras dimensões, adicionamos uma chave substituta para garantir a unicidade de cada registro de geografia e facilitar a integração com outras dimensões e fatos no modelo dimensional.

In [0]:
# 1. Definir o nome da tabela de destino para a dimensão Geografia.
tb_destino = "dim_geografia"  # Nome da tabela de dimensão para dados geográficos.

# 2. Extrair registros únicos para a dimensão Geografia:
# Selecionamos as colunas de localização geográfica (Cidade, Estado, Região, etc.) e garantimos que sejam únicas.
dim_geografia_df = df_silver.select(
    "Cidade", "Estado", "Regiao", "Distrito", "Pais", "CodigoPostal"  # Colunas geográficas necessárias.
).dropDuplicates()  # Remove duplicatas, garantindo que cada combinação de geografia seja única.

# 3. Adicionar chave substituta (surrogate key):
# A chave substituta é gerada para garantir que cada geografia tenha uma identidade única.
dim_geografia_df = dim_geografia_df.withColumn("sk_geografia", monotonically_increasing_id() + 1)

# 4. Escrever a tabela de dimensão no formato Delta:
# Utilizamos o formato Delta para garantir integridade transacional e otimização no desempenho de leitura/escrita.
dim_geografia_df.write.format("delta") \
                 .mode("overwrite") \
                 .save(f"{gold_path}/{tb_destino}")

# Explicação do código acima:
# format("delta"): Define o formato de saída como Delta.
# Substitui os dados existentes, se houver.
# Salva no caminho da camada Gold.

# 5. Contar o número de registros na dimensão:
# Realiza uma contagem para verificar quantas geografias foram processadas.
dim_geografia_df.count()  # Conta o número de registros únicos de geografia na dimensão.


Out[7]: 20513

#### Criação da Dimensão Cliente

Nesta etapa, estamos criando a dimensão Cliente, que armazena informações detalhadas sobre os clientes, como nome, e-mail e dados de localização (cidade, estado, etc.). A criação dessa dimensão permite que as análises de vendas ou outros indicadores possam ser segmentadas por cliente, facilitando o entendimento do comportamento e preferências de compra. Além disso, a dimensão cliente será associada à dimensão Geografia, utilizando a chave substituta (SK) para garantir a integridade dos dados.

In [0]:
# 1. Definir o nome da tabela de destino para a dimensão Cliente.
tb_destino = "dim_cliente"  # Nome da tabela de dimensão para os clientes.

# 2. Importar as funções necessárias do PySpark.
from pyspark.sql.functions import col, monotonically_increasing_id

# 3. Extrair clientes únicos para a dimensão Cliente:
# Selecionamos as colunas necessárias e removemos duplicatas para garantir que cada cliente seja único.
dim_cliente_df = df_silver.select(
    "IDCliente", "Nome", "Email", "Cidade", "Estado", "Regiao", "Distrito", "Pais", "CodigoPostal"  # Informações do cliente.
).dropDuplicates()  # Remove duplicatas, garantindo clientes únicos.

# 4. Realizar o join com a dimensão Geografia para obter a chave substituta (SK_Geografia):
# Associamos a dimensão Cliente à dimensão Geografia, usando os campos de localização (cidade, estado, etc.).
dim_cliente_com_sk_df = dim_cliente_df.alias("cliente") \
    .join(dim_geografia_df.alias("geografia"), 
          (col("cliente.Cidade") == col("geografia.Cidade")) &  # Comparando cidade
          (col("cliente.Estado") == col("geografia.Estado")) &  # Comparando estado
          (col("cliente.Regiao") == col("geografia.Regiao")) &  # Comparando região
          (col("cliente.Distrito") == col("geografia.Distrito")) &  # Comparando distrito
          (col("cliente.Pais") == col("geografia.Pais")) &  # Comparando país
          (col("cliente.CodigoPostal") == col("geografia.CodigoPostal")),  # Comparando código postal
          "left") \
    .select("cliente.IDCliente", "cliente.Nome", "cliente.Email", "geografia.sk_geografia")  # Seleciona as colunas de interesse.

# 5. Adicionar chave substituta (SK_Cliente):
# A chave substituta garante a integridade dos dados e permite vincular a dimensão Cliente com outras tabelas.
dim_cliente_com_sk_df = dim_cliente_com_sk_df.withColumn("sk_cliente", monotonically_increasing_id() + 1)

# 6. Selecionar as colunas finais para a dimensão Cliente:
# Após adicionar a chave substituta, selecionamos as colunas finais a serem salvas na tabela.
dim_cliente_com_sk_df = dim_cliente_com_sk_df.select("IDCliente", "Nome", "Email", "sk_geografia", "sk_cliente")

# 7. Escrever a Dimensão Cliente no formato Delta:
# Salvamos a dimensão Cliente no formato Delta, que oferece transações ACID e desempenho otimizado para leitura e escrita.
dim_cliente_com_sk_df.write.format("delta").mode("overwrite").save(f"{gold_path}/{tb_destino}")

# 8. Contar o número de registros na Dimensão Cliente:
# Realiza uma contagem para verificar quantos clientes foram processados.
dim_cliente_com_sk_df.count()  # Conta o número de registros na dimensão cliente.


Out[8]: 116280

#### Criação de Tabela Fato

Neste processo, estamos criando a tabela Fato de Vendas, que contém as transações de vendas reais, com as chaves substitutas associadas às dimensões previamente criadas (como Produto, Categoria, Segmento, Fabricante, e Cliente). Essa tabela é crucial para análises analíticas de vendas, como a agregação de unidades vendidas, receitas e custos. A tabela será particionada por Ano e Mês para otimizar o desempenho de leitura e escrita em consultas de grandes volumes de dados.

In [0]:
# 1. Definir o nome da tabela de destino para a tabela Fato.
tb_destino = "fato_vendas"  # Nome da tabela que armazena as transações de vendas.

# 2. Importar as funções necessárias do PySpark.
from pyspark.sql.functions import broadcast, year, month

# 3. Juntar dados da camada Silver com as dimensões para obter as chaves substitutas (SKs):
# A operação join é usada para associar a tabela de vendas (df_silver) com as dimensões previamente criadas (Produto, Categoria, Segmento, Fabricante, Cliente).
# O broadcast é utilizado para otimizar a junção, distribuindo eficientemente as tabelas menores.
fato_vendas_df = df_silver.alias("s") \
    .join(broadcast(dim_produto_df.select("IDProduto", "sk_produto").alias("dprod")), "IDProduto") \
    .join(broadcast(dim_categoria_df.select("Categoria", "sk_categoria").alias("dcat")), "Categoria") \
    .join(broadcast(dim_segmento_df.select("Segmento", "sk_segmento").alias("dseg")), "Segmento") \
    .join(broadcast(dim_fabricante_df.select("Fabricante", "sk_fabricante").alias("dfab")), "Fabricante") \
    .join(broadcast(dim_cliente_com_sk_df.select("IDCliente", "sk_cliente").alias("dcli")), "IDCliente") \
    .select(  # Seleciona as colunas de interesse para a tabela Fato.
        col("s.Data").alias("DataVenda"),  # Data da venda
        "sk_produto",  # Chave substituta para o Produto
        "sk_categoria",  # Chave substituta para a Categoria
        "sk_segmento",  # Chave substituta para o Segmento
        "sk_fabricante",  # Chave substituta para o Fabricante
        "sk_cliente",  # Chave substituta para o Cliente
        "Unidades",  # Número de unidades vendidas
        col("s.PrecoUnitario"),  # Preço unitário
        col("s.CustoUnitario"),  # Custo unitário
        col("s.TotalVendas")  # Valor total da venda
    )

# 4. Escrever a tabela Fato no formato Delta:
# A tabela Fato será salva no formato Delta, o que garante transações ACID, desempenho otimizado e a capacidade de fazer upserts.
# A tabela será particionada por Ano e Mês para otimizar consultas baseadas em datas.
fato_vendas_df.withColumn("Ano", year("DataVenda")) \
             .withColumn("Mes", month("DataVenda")) \
             .write.format("delta") \
             .mode("overwrite") \
             .option("MaxRecordsPerFile", 1000000) \
             .partitionBy("Ano", "Mes") \
             .save(f"{gold_path}/{tb_destino}")

# Explicação do código acima
# withColumn("Ano", year("DataVenda")): Criação da coluna "Ano"
# withColumn("Mes", month("DataVenda")): Criação da coluna "Mes"
# format("delta"): Usando formato Delta para transações ACID
# mode("overwrite"): Modo overwrite para sobrescrever dados existentes
# MaxRecordsPerFile: Definir limite de registros por arquivo para evitar arquivos muito grandes
# partitionBy("Ano", "Mes") : Particionar dados por Ano e Mês para otimizar consultas
# save(f"{gold_path}/{tb_destino}"): Salvar a tabela Fato no caminho especificado.

# 5. Contar o número de registros na Tabela Fato:
# Realiza uma contagem para verificar quantos registros de vendas foram processados.
fato_vendas_df.count()  # Contagem do número de registros na tabela Fato.


Out[10]: 116895

### Limpeza de Memória

Após realizar operações pesadas de processamento de dados, como junções complexas, leituras e gravações, é importante liberar memória para garantir que o ambiente do Spark continue eficiente e sem sobrecarregar o sistema. O código abaixo é usado para realizar a limpeza de memória de forma eficaz.

In [0]:
# 1. Importação da biblioteca para gerenciamento de memória
import gc  # gc (Garbage Collector) é uma biblioteca que permite gerenciar a memória, coletando objetos não utilizados.

# 2. Coletar lixo (garbage collection) para liberar memória após operações pesadas
gc.collect()  # Coleta os objetos que não estão mais em uso na memória e os remove, liberando espaço.

# 3. Limpar todos os dados em cache do Spark
spark.catalog.clearCache()  # Limpa todos os dados armazenados em cache pelo Spark, liberando memória.

###Verificação dos arquivos gerados

In [0]:
%fs ls /mnt/lhdw/gold/vendas_delta/

path,name,size,modificationTime
dbfs:/mnt/lhdw/gold/vendas_delta/dim_categoria/,dim_categoria/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/dim_cliente/,dim_cliente/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/dim_fabricante/,dim_fabricante/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/dim_geografia/,dim_geografia/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/dim_produto/,dim_produto/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/dim_segmento/,dim_segmento/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas/,fato_vendas/,0,0


In [0]:
%fs ls /mnt/lhdw/gold/vendas_delta/dim_categoria/

path,name,size,modificationTime
dbfs:/mnt/lhdw/gold/vendas_delta/dim_categoria/_delta_log/,_delta_log/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/dim_categoria/part-00000-2eb98646-bf3d-41a8-9a7d-deb96fcf3d8b-c000.snappy.parquet,part-00000-2eb98646-bf3d-41a8-9a7d-deb96fcf3d8b-c000.snappy.parquet,948,1733235056000
dbfs:/mnt/lhdw/gold/vendas_delta/dim_categoria/part-00000-823ace36-c270-40c8-8ac2-5c8e532a3c84-c000.snappy.parquet,part-00000-823ace36-c270-40c8-8ac2-5c8e532a3c84-c000.snappy.parquet,948,1734114425000


In [0]:
%fs ls /mnt/lhdw/gold/vendas_delta/fato_vendas/Ano=2012/

path,name,size,modificationTime
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas/Ano=2012/Mes=1/,Mes=1/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas/Ano=2012/Mes=10/,Mes=10/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas/Ano=2012/Mes=11/,Mes=11/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas/Ano=2012/Mes=12/,Mes=12/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas/Ano=2012/Mes=2/,Mes=2/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas/Ano=2012/Mes=3/,Mes=3/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas/Ano=2012/Mes=4/,Mes=4/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas/Ano=2012/Mes=5/,Mes=5/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas/Ano=2012/Mes=6/,Mes=6/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas/Ano=2012/Mes=7/,Mes=7/,0,0
